In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os 
while 'notebooks' in os.getcwd():
    os.chdir("..")
    
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import csv
from tqdm import tqdm
import missingno as msno

import warnings
warnings.filterwarnings("ignore")
from src.constants import CLEANED_HOUSEHOLD_POWER_PATH, CLEANED_WEATHER_PATH, \
    HOUSEHOLD_WEATHER_MERGED_PATH

## Merging datasets

In [3]:
household_consumption_df = pd.read_pickle(
    CLEANED_HOUSEHOLD_POWER_PATH, 
)

household_consumption_df

,global_active_power,global_reactive_power,voltage,global_intensity,sub_metering_1,sub_metering_2,sub_metering_3
date,,,,,,,
2007-01-01 00:00:00,2.580,0.136,241.97,10.6,0.0,0.0,0.0
2007-01-01 00:01:00,2.552,0.100,241.75,10.4,0.0,0.0,0.0
2007-01-01 00:02:00,2.550,0.100,241.64,10.4,0.0,0.0,0.0
2007-01-01 00:03:00,2.550,0.100,241.71,10.4,0.0,0.0,0.0
2007-01-01 00:04:00,2.554,0.100,241.98,10.4,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2010-11-26 20:58:00,0.946,0.000,240.43,4.0,0.0,0.0,0.0
2010-11-26 20:59:00,0.944,0.000,240.00,4.0,0.0,0.0,0.0
2010-11-26 21:00:00,0.938,0.000,239.82,3.8,0.0,0.0,0.0


In [4]:
weather_df = pd.read_pickle(
    CLEANED_WEATHER_PATH,
)

weather_df

,numer_sta,pmer,tend,cod_tend,dd,ff,t,td,u,vv,...,phenspe1,phenspe2,phenspe3,nnuage1,ctype1,hnuage1,Nom,Latitude,Longitude,Altitude
date,,,,,,,,,,,,,,,,,,,,,
2007-01-01 00:00:00,7149,101960.0,-70.0,6.0,220.0,9.8,286.65,284.85,89.0,20000.0,...,1017.0,710.0,1118.0,2.0,8.0,780.0,ORLY,48.716833,2.384333,89.0
2007-01-01 03:00:00,7149,102010.0,40.0,3.0,240.0,7.2,285.65,282.25,80.0,20000.0,...,710.0,1114.0,1116.5,2.0,8.0,780.0,ORLY,48.716833,2.384333,89.0
2007-01-01 06:00:00,7149,102040.0,30.0,3.0,230.0,8.7,283.65,279.75,77.0,20000.0,...,1015.0,710.0,1115.0,2.0,8.0,780.0,ORLY,48.716833,2.384333,89.0
2007-01-01 09:00:00,7149,102240.0,190.0,3.0,250.0,9.8,282.05,278.05,76.0,17000.0,...,1018.0,710.0,1118.0,2.0,8.0,450.0,ORLY,48.716833,2.384333,89.0
2007-01-01 12:00:00,7149,102450.0,210.0,0.0,270.0,8.7,283.65,276.65,62.0,25000.0,...,710.0,1114.0,3100.0,1.0,8.0,840.0,ORLY,48.716833,2.384333,89.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31 09:00:00,7149,102580.0,140.0,1.0,90.0,4.1,273.45,273.05,97.0,3000.0,...,710.0,1106.0,3100.0,3.0,7.0,60.0,ORLY,48.716833,2.384333,89.0
2010-12-31 12:00:00,7149,102540.0,-40.0,8.0,40.0,4.1,274.05,273.45,96.0,7000.0,...,710.0,1106.0,3100.0,4.0,7.0,90.0,ORLY,48.716833,2.384333,89.0
2010-12-31 15:00:00,7149,102540.0,0.0,5.0,80.0,3.1,274.15,272.95,92.0,13000.0,...,710.0,1105.0,3100.0,4.0,7.0,180.0,ORLY,48.716833,2.384333,89.0


In [5]:
weather_df.loc['2010-12-11'].index

DatetimeIndex(['2010-12-11 00:00:00', '2010-12-11 03:00:00',
               '2010-12-11 06:00:00', '2010-12-11 09:00:00',
               '2010-12-11 12:00:00', '2010-12-11 15:00:00',
               '2010-12-11 18:00:00', '2010-12-11 21:00:00'],
              dtype='datetime64[ns]', name='date', freq=None)

In [7]:
weather_df.loc[:'2010-12-11'].index.isin(household_consumption_df.index)

array([ True,  True,  True, ..., False, False, False])

In [8]:
df = pd.merge(
    household_consumption_df,
    weather_df,
    how = 'left',
    left_index=True,
    right_index=True
)

In [9]:
df[["global_active_power",	"global_reactive_power",	"voltage",	"global_intensity",	"sub_metering_1",	"sub_metering_2",	"sub_metering_3"]].isna().sum()

global_active_power      0
global_reactive_power    0
voltage                  0
global_intensity         0
sub_metering_1           0
sub_metering_2           0
sub_metering_3           0
dtype: int64

In [10]:
df.fillna(method='ffill')\
    .to_pickle(
        HOUSEHOLD_WEATHER_MERGED_PATH
    )